In [7]:
%reload_ext autoreload
%autoreload 2

import os
import pickle
import logging
import multiprocessing
from pathlib import Path

import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

import arviz as az
import numpyro

from hbmep.config import Config
from hbmep.model.utils import Site as site

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)

In [8]:
import numpyro.distributions as dist
from hbmep.model import Baseline


class RectifiedLogistic(Baseline):
    LINK = "rectified_logistic"

    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]

    def fn(self, x, a, b, v, l, L, H):
        return(
            L
            + jnp.where(
                x <= a,
                0.,
                -l
                + (
                    (H + l)
                    / jnp.power(
                        1
                        + (
                            (
                                -1
                                + jnp.power(
                                    (H + l) / l,
                                    v
                                )
                            )
                            * jnp.exp(-b * (x - a))
                        ),
                        1 / v
                    )
                )
            )
        )

    def fn_prime(self, x, *args):
        fn_grad = jax.grad(self.fn, argnums=0)
        for _ in range(len(x.shape)): fn_grad = jax.vmap(fn_grad)
        return fn_grad(x, *args)

    def _model(self, subject, features, intensity, response_obs=None):
        subject, n_subject = subject
        features, n_features = features
        intensity, n_data = intensity

        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)
        n_feature0 = n_features[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            global_sigma_b = numpyro.sample("global_sigma_b", dist.HalfNormal(100))
            global_sigma_v = numpyro.sample("global_sigma_v", dist.HalfNormal(100))

            global_sigma_L = numpyro.sample("global_sigma_L", dist.HalfNormal(1))
            global_sigma_l = numpyro.sample("global_sigma_l", dist.HalfNormal(100))
            global_sigma_H = numpyro.sample("global_sigma_H", dist.HalfNormal(5))

            global_sigma_g_1 = numpyro.sample("global_sigma_g_1", dist.HalfNormal(100))
            global_sigma_g_2 = numpyro.sample("global_sigma_g_2", dist.HalfNormal(100))

            global_sigma_p = numpyro.sample("global_sigma_p", dist.HalfNormal(100))

            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(site.mu_a, dist.HalfNormal(scale=5))
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(scale=1 / 10))

                sigma_b_raw = numpyro.sample("sigma_b_raw", dist.HalfNormal(scale=1))
                sigma_b = numpyro.deterministic(site.sigma_b, global_sigma_b * sigma_b_raw)

                sigma_v_raw = numpyro.sample("sigma_v_raw", dist.HalfNormal(scale=1))
                sigma_v = numpyro.deterministic(site.sigma_v, global_sigma_v * sigma_v_raw)

                sigma_L_raw = numpyro.sample("sigma_L_raw", dist.HalfNormal(scale=1))
                sigma_L = numpyro.deterministic(site.sigma_L, global_sigma_L * sigma_L_raw)

                sigma_l_raw = numpyro.sample("sigma_l_raw", dist.HalfNormal(scale=1))
                sigma_l = numpyro.deterministic("sigma_l", global_sigma_l * sigma_l_raw)

                sigma_H_raw = numpyro.sample("sigma_H_raw", dist.HalfNormal(scale=1))
                sigma_H = numpyro.deterministic(site.sigma_H, global_sigma_H * sigma_H_raw)

                sigma_g_1_raw = numpyro.sample("sigma_g_1_raw", dist.HalfNormal(scale=1))
                sigma_g_1 = numpyro.deterministic("sigma_g_1", global_sigma_g_1 * sigma_g_1_raw)

                sigma_g_2_raw = numpyro.sample("sigma_g_2_raw", dist.HalfNormal(scale=1))
                sigma_g_2 = numpyro.deterministic("sigma_g_2", global_sigma_g_2 * sigma_g_2_raw)

                sigma_p_raw = numpyro.sample("sigma_p_raw", dist.HalfNormal(scale=1))
                sigma_p = numpyro.deterministic("sigma_p", global_sigma_p * sigma_p_raw)

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    """ Priors """
                    a_raw = numpyro.sample("a_raw", dist.Gamma(concentration=mu_a, rate=1))
                    a = numpyro.deterministic(site.a, (1 / sigma_a) * a_raw)

                    b_raw = numpyro.sample("b_raw", dist.HalfNormal(scale=1))
                    b = numpyro.deterministic(site.b, sigma_b * b_raw)

                    v_raw = numpyro.sample("v_raw", dist.HalfNormal(scale=1))
                    v = numpyro.deterministic(site.v, sigma_v * v_raw)

                    L_raw = numpyro.sample("L_raw", dist.HalfNormal(scale=1))
                    L = numpyro.deterministic(site.L, sigma_L * L_raw)

                    l_raw = numpyro.sample("l_raw", dist.HalfNormal(scale=1))
                    l = numpyro.deterministic("l", sigma_l * l_raw)

                    H_raw = numpyro.sample("H_raw", dist.HalfNormal(scale=1))
                    H = numpyro.deterministic(site.H, sigma_H * H_raw)

                    g_1_raw = numpyro.sample("g_1_raw", dist.HalfCauchy(scale=1))
                    g_1 = numpyro.deterministic(site.g_1, sigma_g_1 * g_1_raw)

                    g_2_raw = numpyro.sample("g_2_raw", dist.HalfCauchy(scale=1))
                    g_2 = numpyro.deterministic(site.g_2, sigma_g_2 * g_2_raw)

                    p_raw = numpyro.sample("p_raw", dist.HalfNormal(scale=1))
                    p = numpyro.deterministic("p", sigma_p * p_raw)

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.data, n_data, dim=-2):
                """ Model """
                mu = numpyro.deterministic(
                    site.mu,
                    self.fn(
                        intensity,
                        a[subject, feature0],
                        b[subject, feature0],
                        v[subject, feature0],
                        l[subject, feature0],
                        L[subject, feature0],
                        H[subject, feature0]
                    )
                )
                mu_prime = numpyro.deterministic(
                    "mu_prime",
                    self.fn_prime(
                        intensity,
                        a[subject, feature0],
                        b[subject, feature0],
                        v[subject, feature0],
                        l[subject, feature0],
                        L[subject, feature0],
                        H[subject, feature0]
                    )
                )
                # beta = numpyro.deterministic(
                #     site.beta,
                #     g_1[subject, feature0] + g_2[subject, feature0] * jnp.power(1 / (mu_prime + 1), p[subject, feature0])
                # )
                beta = numpyro.deterministic(
                    site.beta,
                    g_1[subject, feature0] + g_2[subject, feature0] * (1 / (mu_prime + 1))
                )
                # mu = numpyro.deterministic(
                #     site.mu,
                #     L[subject, feature0]
                #     + jnp.where(
                #         intensity <= a[subject, feature0],
                #         0,
                #         -l[subject, feature0]
                #         + (
                #             (H[subject, feature0] + l[subject, feature0])
                #             / jnp.power(
                #                 1
                #                 + (
                #                     (
                #                         -1
                #                         + jnp.power(
                #                             (H[subject, feature0] + l[subject, feature0]) / l[subject, feature0],
                #                             v[subject, feature0]
                #                         )
                #                     )
                #                     * jnp.exp(-b[subject, feature0] * (intensity - a[subject, feature0]))
                #                 ),
                #                 1 / v[subject, feature0]
                #             )
                #         )
                #     )
                # )
                # beta = numpyro.deterministic(
                #     site.beta,
                #     g_1[subject, feature0] + g_2[subject, feature0] * jnp.power(1 / (mu + 1), p[subject, feature0])
                # )

                """ Observation """
                numpyro.sample(
                    site.obs,
                    dist.Gamma(concentration=mu * beta, rate=beta),
                    obs=response_obs
                )


In [9]:
toml_path = "/home/vishu/repos/hbmep-paper/configs/paper/tms/link-comparison/proc-2023-10-27/injury/rectified_logistic.toml"

config = Config(toml_path=toml_path)
config.MCMC_PARAMS["num_warmup"] = 500
config.MCMC_PARAMS["num_samples"] = 1000
# config.MCMC_PARAMS["thinning"] = 4

model = RectifiedLogistic(config=config)

# from numpyro.infer import NUTS
# sampler = NUTS(model=model._model, max_tree_depth=50)


2023-11-01 15:20:05,014 - hbmep.config - INFO - Verifying configuration ...
2023-11-01 15:20:05,014 - hbmep.config - INFO - Success!
2023-11-01 15:20:05,015 - hbmep.model.baseline - INFO - Initialized model with rectified_logistic link


In [10]:
src = "/home/vishu/data/hbmep-processed/human/tms/data_pkpk_auc.csv"
df = pd.read_csv(src)

subset = ["SCA01"]
ind = df[model.subject].isin(subset)
df = df[ind].reset_index(drop=True).copy()

df, encoder_dict = model.load(df=df)


2023-11-01 15:20:05,248 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/proc-2023-10-27/injury


2023-11-01 15:20:05,295 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/proc-2023-10-27/injury
2023-11-01 15:20:05,299 - hbmep.dataset.core - INFO - Processing data ...
2023-11-01 15:20:05,302 - hbmep.utils.utils - INFO - func:load took: 0.05 sec


In [11]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-11-01 15:20:05,402 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

2023-11-01 15:20:30,299 - hbmep.utils.utils - INFO - func:run_inference took: 24.90 sec


In [12]:
prediction_df = model.make_prediction_dataset(df=df)
posterior_predictive = model.predict(df=prediction_df, posterior_samples=posterior_samples)

model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples, prediction_df=prediction_df, posterior_predictive=posterior_predictive)
model.render_predictive_check(df=df, encoder_dict=encoder_dict, prediction_df=prediction_df, posterior_predictive=posterior_predictive)


2023-11-01 15:20:30,426 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.00 sec


2023-11-01 15:20:31,135 - hbmep.utils.utils - INFO - func:predict took: 0.71 sec
2023-11-01 15:20:31,137 - hbmep.model.baseline - INFO - Rendering ...
2023-11-01 15:20:31,331 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/proc-2023-10-27/injury/recruitment_curves.pdf
2023-11-01 15:20:31,331 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 0.20 sec
2023-11-01 15:20:31,333 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...
2023-11-01 15:20:31,539 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/proc-2023-10-27/injury/posterior_predictive_check.pdf
2023-11-01 15:20:31,539 - hbmep.utils.utils - INFO - func:_render_predictive_check took: 0.21 sec
2023-11-01 15:20:31,540 - hbmep.utils.utils - INFO - func:render_predictive_check took: 0.21 sec


In [11]:
posterior_samples["mu_prime"].

0

In [13]:
mcmc.print_summary(prob=.95)



                         mean       std    median      2.5%     97.5%     n_eff     r_hat
       H_raw[0,0,0]      0.62      0.46      0.48      0.04      1.59     98.48      1.02
       L_raw[0,0,0]      0.50      0.54      0.29      0.01      1.76     97.28      1.04
       a_raw[0,0,0]      3.88      1.95      3.50      0.80      7.63     59.75      1.07
       b_raw[0,0,0]      0.54      0.54      0.36      0.00      1.75     60.34      1.03
     g_1_raw[0,0,0]      1.42      6.83      0.30      0.00      3.19    107.31      1.04
     g_2_raw[0,0,0]      0.92      1.96      0.47      0.01      3.25    155.34      1.02
  global_sigma_H[0]      3.60      2.44      3.05      0.37      8.44     88.48      1.03
  global_sigma_L[0]      0.61      0.50      0.52      0.01      1.50     82.33      1.07
  global_sigma_b[0]     52.80     50.85     35.97      0.56    155.35    102.68      1.01
global_sigma_g_1[0]     36.48     43.82     20.28      0.00    127.83    136.90      1.02
global_si

In [ ]:
numpyro_data = az.from_numpyro(mcmc)

""" Model evaluation """
logger.info("Evaluating model ...")

score = az.loo(numpyro_data)
logger.info(f"ELPD LOO (Log): {score.elpd_loo:.2f}")

score = az.waic(numpyro_data)
logger.info(f"ELPD WAIC (Log): {score.elpd_waic:.2f}")


2023-10-27 15:51:53,418 - __main__ - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1037: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
2023-10-27 15:51:59,497 - __main__ - INFO - ELPD LOO (Log): 9011.30
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities

In [ ]:
import pickle

dest = os.path.join(model.build_dir, "inference.pkl")
with open(dest, "wb") as f:
    pickle.dump((model, mcmc, posterior_samples), f)


In [ ]:
dest = os.path.join(model.build_dir, "numpyro_data.nc")
az.to_netcdf(numpyro_data, dest)


'/home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/rectified_logistic/numpyro_data.nc'